In [1]:
import sys, os, time
import shutil
%pdb
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision

from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
# %matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import imageio
import seaborn as sns
# plt.switch_backend('Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from bisect import bisect_left

# from IPython.display import HTML, Image

#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data
from analyze import *

from tqdm import tqdm_notebook as tqdm

# plt.ioff()

np.set_printoptions(linewidth=120, suppress=True)

Automatic pdb calling has been turned ON


In [2]:
media_dir = '/is/ei/fleeb/workspace/media/hybrid/results'

In [3]:
M = Hybrid_Controller(media_dir).sort_by('date')

Found 122 runs


In [4]:
M.clear_filters()
pass

In [5]:
M.prep_info(name='model.pth.tar')

M.load_into('eval.pth.tar')

Selecting checkpoint: last


/is/ei/fleeb/workspace/foundation/foundation/train/config.py:66: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Loaded configs


In [6]:
def _get_stats(run):
    if 'eval' in run:
        run.stats = {
            'FID-prior': run.eval['FID-prior'],
            
            
#             'DCI-inf': run.eval['DCI']['informativeness_test'],
#             'DCI-cmp': run.eval['DCI']['completeness'],
        }
        
        if 'FID-hyb' in run.eval and run.eval['FID-hyb'] is not None:
            run.stats.update({
                'FID-hyb': run.eval['FID-hyb'],
                'FID-rec': run.eval['FID-rec'],
            })
        
        if 'IRS' in run.eval and run.eval['IRS'] is not None:
            run.stats.update({
                
                'IRS': run.eval['IRS']['IRS'],
                'DCI-dis': run.eval['DCI']['disentanglement'],
            })
M.map(_get_stats, safe=True)

M.filter(lambda r: 'stats' in r and len(r.stats))
pass

M.store_selection('done')


In [7]:
M.show('all')

  0) 3dshapes-dvae_0100-5966131-07_200109-133026_ckpt100
	Checkpoint: 100
	 FID-prior : 152.06
	   FID-hyb : 69.78
	   FID-rec : 26.00
	       IRS : 0.55
	   DCI-dis : 0.55
	model.latent_dim - 8 (16)
	model.generator.full_latent_dim - 8 (_)
	model.prob - 1 (0.5)

  1) 3dshapes-dvae_0100-5966131-08_200109-133040_ckpt16
	Checkpoint: 16
	 FID-prior : 78.53
	   FID-hyb : 54.59
	   FID-rec : 44.36
	       IRS : 0.64
	   DCI-dis : 0.70
	model.latent_dim - 8 (16)
	model.generator.full_latent_dim - 8 (_)

  2) 3dshapes-dvae_0102-5966133-07_200109-133118_ckpt100
	Checkpoint: 100
	 FID-prior : 99.53
	   FID-hyb : 62.09
	   FID-rec : 22.09
	       IRS : 0.66
	   DCI-dis : 0.70
	model.generator.splits - 4 (2)
	model.generator.adain_latent_dim - 16 (8)
	model.generator.full_latent_dim - 16 (_)
	model.prob - 1 (0.5)

  3) 3dshapes-dvae_0102-5966133-08_200109-133107_ckpt100
	Checkpoint: 100
	 FID-prior : 78.20
	   FID-hyb : 47.16
	   FID-rec : 21.70
	       IRS : 0.72
	   DCI-dis : 0.76
	model.genera

In [8]:
M.switch_selection('done')
len(M)

122

In [9]:
# M.invert()
# len(M)
# M.show()
# ' '.join(['_'.join(run.name.split('_')[:-1]) for run in M])

In [10]:
plt.close('all')
figaxs = {}

In [11]:
M.map(lambda r: r.meta.job[0], reduce=Counter)

Counter({'100': 12,
         '102': 11,
         '122': 4,
         '130': 3,
         '131': 4,
         '94': 6,
         '170': 2,
         '174': 14,
         '168': 6,
         '169': 8,
         '138': 6,
         '171': 1,
         '172': 1,
         '178': 6,
         '177': 15,
         '132': 10,
         '136': 13})

In [26]:
M.switch_selection('done')

M.filter(lambda r: 'IRS' in r.stats)

M.sort_by(lambda runs: util.sort_by(runs, [run.stats['IRS'] for run in runs]))
# M.filter_sel(slice(-10,None,None))


M.filter_models('fdwae')
M.filter_jobs(177)
# M.show('checkpoint', 'unique', )
# M.filter(lambda r: r.stats['DCI-dis'] > 0.75)

pass

In [27]:
fig_stats = {
    'dis':['DCI-dis', 'IRS'], 
    'fid2': ['FID-rec', 'FID-hyb'],
    'fid': ['FID-prior', ],
#              'fid':['FID-rec', ],
            }
for i, (name,stats,) in enumerate(fig_stats.items()):
    if name not in figaxs:
        figaxs[name] = plt.subplots(figsize=(12,5))
    figaxs[name][1].cla()
    figaxs[name] = M.plot_stats(*stats, figax=figaxs.get(name,None))#, color='C{}'.format(i))
    plt.tight_layout()
# M.show('checkpoint', 'unique', manuals=[lambda r: 'Generator: {}'.format(r.config.model.generator._type)])
M.show('all', manuals=[lambda r: 'Generator: {}'.format(r.config.model.generator._type)])

  0) nocap-fdwae_0177-6015622-10_200124-222341_ckpt86
	Checkpoint: 86
	 FID-prior : 58.54
	   FID-hyb : 50.41
	   FID-rec : 44.12
	       IRS : 0.59
	   DCI-dis : 0.35
	model.latent_dim - 12 (16)
	model.latent_reg_wt - 0 (100)
	Generator: double-dec

  1) nocap-fdwae_0177-6015622-10_200124-222341_ckpt83
	Checkpoint: 83
	 FID-prior : 58.62
	   FID-hyb : 50.74
	   FID-rec : 44.36
	       IRS : 0.59
	   DCI-dis : 0.34
	model.latent_dim - 12 (16)
	model.latent_reg_wt - 0 (100)
	Generator: double-dec

  2) nocap-fdwae_0177-6015622-10_200124-222341_ckpt55
	Checkpoint: 55
	 FID-prior : 71.26
	   FID-hyb : 68.78
	   FID-rec : 62.25
	       IRS : 0.61
	   DCI-dis : 0.30
	model.latent_dim - 12 (16)
	model.latent_reg_wt - 0 (100)
	Generator: double-dec

  3) nocap-fdwae_0177-6015622-11_200124-222419_ckpt100
	Checkpoint: 100
	 FID-prior : 50.98
	   FID-hyb : 49.47
	   FID-rec : 43.75
	       IRS : 0.67
	   DCI-dis : 0.49
	model.latent_dim - 12 (16)
	model.latent_reg_wt - 1000 (100)
	model.reg_prio

In [23]:
run = M[0]

In [ ]:
run.config.model.generator._type